In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
import os
os.chdir("drive/MyDrive/data")

Mounted at /content/drive/


In [ ]:
!ls

emotion_labels.json	    final_train_tweets.json  sentiment_test_tweet.json
emotion_test_tweet.json     fixed_train_tweets.json  title_conference.csv
final_test_candidates.json  model		     topic_labels.json
final_test_tweets.json	    outp		     topic_test_tweet0.json
final_train_labels.json     sentiment_labels.json    topic_test_tweet.json


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 7.5MB/s 
     |████████████████████████████████| 3.3MB 35.3MB/s 
     |████████████████████████████████| 901kB 37.6MB/s 


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import pandas as pd
import numpy as np


In [ ]:
df_tls = pd.read_json("final_train_labels.json", orient='records', dtype=False)
print(df_tls.shape)
df_tls.head()

(2000, 3)


,tweet_id,m_id,m_label
0,1345642006299594752,7,agree
1,1339975577701392385,4,no_stance
2,1345867896426278913,10,agree
3,1344512357427511297,1,not_relevant
4,1340568636201848832,8,disagree


In [ ]:
label_dict = {'agree': 0, 'disagree': 1, 'no_stance': 2, 'not_relevant': 3}
label_dict_inverse = {v: k for k, v in label_dict.items()}
df_tls['label'] = df_tls.m_label.replace(label_dict)
df_tls.head()

,tweet_id,m_id,m_label,label
0,1345642006299594752,7,agree,0
1,1339975577701392385,4,no_stance,2
2,1345867896426278913,10,agree,0
3,1344512357427511297,1,not_relevant,3
4,1340568636201848832,8,disagree,1


In [ ]:
df_tts = pd.read_json("final_train_tweets.json", orient='records', dtype=False)
df_tts.rename(columns={'id':'tweet_id'}, inplace = True)
print(df_tts.shape)
df_tts.head()

(1640, 2)


,tweet_id,text
0,1344053526951985152,Human immune system has a higher success rate ...
1,1340150211423952896,Covid Vaccine is the new version of population...
2,1340070783536812034,Do not take the COVID-19 vaccine. You will nev...
3,1343310998485094400,@stealthbadger @oneconnelly1 Something in the ...
4,1343303464697270273,"The Pope of Rome declares that ""the use of the..."


In [ ]:
df = pd.merge(df_tls, df_tts, how='left', on=['tweet_id'])
print(df.shape)
df.head()

(2000, 5)


,tweet_id,m_id,m_label,label,text
0,1345642006299594752,7,agree,0,Foetal cells are used to make the Oxford coron...
1,1339975577701392385,4,no_stance,2,They were just impersonating Tyler Herro URL Q...
2,1345867896426278913,10,agree,0,@RMCpost @SitaramYechury The AstraZeneca vacci...
3,1344512357427511297,1,not_relevant,3,This will happen if you take the Covid-19 vacc...
4,1340568636201848832,8,disagree,1,The COVID vaccine will not contain a microchip...


In [ ]:
def discretize(value, length):
    l = ["0"] * (length - 1)
    l[value - 1] = "1"
    return " ".join(l)

In [ ]:
possible_m_ids = df.m_id.unique()

m_id_dict = {}
for index, possible_m_id in enumerate(possible_m_ids):
    m_id_dict[possible_m_id] = index
m_id_dict

{'1': 3, '10': 2, '2': 5, '3': 6, '4': 1, '7': 0, '8': 4, '9': 7}

In [ ]:
df['input'] = df['m_id'].map(lambda x: discretize(m_id_dict[x], len(m_id_dict)) + " ") + df['text']
df.head()

,tweet_id,m_id,m_label,label,text,input
0,1345642006299594752,7,agree,0,Foetal cells are used to make the Oxford coron...,0 0 0 0 0 0 1 Foetal cells are used to make th...
1,1339975577701392385,4,no_stance,2,They were just impersonating Tyler Herro URL Q...,1 0 0 0 0 0 0 They were just impersonating Tyl...
2,1345867896426278913,10,agree,0,@RMCpost @SitaramYechury The AstraZeneca vacci...,0 1 0 0 0 0 0 @RMCpost @SitaramYechury The Ast...
3,1344512357427511297,1,not_relevant,3,This will happen if you take the Covid-19 vacc...,0 0 1 0 0 0 0 This will happen if you take the...
4,1340568636201848832,8,disagree,1,The COVID vaccine will not contain a microchip...,0 0 0 1 0 0 0 The COVID vaccine will not conta...


In [ ]:
df['m_label'].value_counts()

not_relevant    783
agree           608
disagree        354
no_stance       255
Name: m_label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_trn, X_val, y_trn, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.3, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_trn, 'data_type'] = 'trn'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['m_label', 'label', 'data_type']).count()

tweet_id  m_id  text  input
m_label      label data_type                             
agree        0     trn             426   426   426    426
                   val             182   182   182    182
disagree     1     trn             248   248   248    248
                   val             106   106   106    106
no_stance    2     trn             178   178   178    178
                   val              77    77    77     77
not_relevant 3     trn             548   548   548    548
                   val             235   235   235    235

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
encoded_data_trn = tokenizer.batch_encode_plus(
    df[df.data_type=='trn'].input.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    truncation=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].input.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    truncation=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_trn = encoded_data_trn['input_ids']
attention_masks_trn = encoded_data_trn['attention_mask']
labels_trn = torch.tensor(df[df.data_type=='trn'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_trn = TensorDataset(input_ids_trn, attention_masks_trn, labels_trn)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

len(dataset_trn), len(dataset_val)

(1400, 600)

In [ ]:
print("Before encoding.")
print(df.input.values[0:3])
print("After encoding.")
print(input_ids_trn.numpy()[0:3])

Before encoding.
['0 0 0 0 0 0 1 Foetal cells are used to make the Oxford coronavirus vaccine. But they came from a foetus in 1973 - ABC News URL'
 '1 0 0 0 0 0 0 They were just impersonating Tyler Herro URL QT: "Four trial volunteers who got Pfizer\'s COVID-19 vaccine developed Bell\'s palsy - but FDA denies that the temporary facial paralysis was caused by the shot.\nURL URL"'
 '0 1 0 0 0 0 0 @RMCpost @SitaramYechury The AstraZeneca vaccine was tested but had serious side effects worse than having COVID. So tell me why inject these in the first place?']
After encoding.
[[  101  1014  1014  1014  1014  1014  1014  1015 22277  9080  4442  2024
   2109  2000  2191  1996  4345 21887 23350 17404  1012  2021  2027  2234
   2013  1037 22277  5809  1999  3381  1011  5925  2739 24471  2140   102
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0   

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_trn, 
                              sampler=RandomSampler(dataset_trn), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    preds_sum, true_sum = 0, 0

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}  Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
        true_sum += len(y_preds[y_preds==label])
        preds_sum += len(y_true)
    
    total_accuracy = "%.4f" % (true_sum / preds_sum * 100)
    print(f"Total accuracy: {total_accuracy}%\n")
    return float(total_accuracy)

In [ ]:
epochs = 20

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'model/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 1.178508349225659
Validation loss: 1.0555111226439475
F1 Score (Weighted): 0.4704117439787754



Epoch 2
Training loss: 1.0006690585559006
Validation loss: 0.9910619774647057
F1 Score (Weighted): 0.5918168985287847



Epoch 3
Training loss: 0.793841526586347
Validation loss: 1.1404854027694091
F1 Score (Weighted): 0.5749137070433699



Epoch 4
Training loss: 0.596893476581548
Validation loss: 1.3089041114877908
F1 Score (Weighted): 0.6470689271968343



Epoch 5
Training loss: 0.4873071758832376
Validation loss: 1.5194088109373114
F1 Score (Weighted): 0.6473786220357199


...


Epoch 16
Training loss: 0.09052054242364817
Validation loss: 2.669734837290962
F1 Score (Weighted): 0.6520541366896477



Epoch 17
Training loss: 0.08745459925857092
Validation loss: 2.687594720635534
F1 Score (Weighted): 0.6550872844744372



Epoch 18
Training loss: 0.07715385025129698
Validation loss: 2.6371790080281787
F1 Score (Weighted): 0.6576070288592675



Epoch 19
Training loss: 0.06854940121470551
Validation loss: 2.639777201030156
F1 Score (Weighted): 0.6530514303595581



Epoch 20
Training loss: 0.06459551766905904
Validation loss: 2.6516180362720116
F1 Score (Weighted): 0.6502993063366547



In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
best_dir = ""
highest_acc = 0
for epoch in tqdm(range(1, epochs+1)):
    print(f"Epoch {epoch}:")
    dir = f"model/finetuned_BERT_epoch_{epoch}.model"
    model.load_state_dict(torch.load(dir, map_location=torch.device('cpu')))
    _, predictions, true_vals = evaluate(dataloader_validation)
    acc = accuracy_per_class(predictions, true_vals)
    if acc > highest_acc:
        highest_acc = acc
        best_dir = dir
        print("Temporary best.\n")
print(f"Best model: {best_dir}")
model.load_state_dict(torch.load(best_dir, map_location=torch.device('cpu')))

Epoch 1:
Class: agree  Accuracy: 156/182
Class: disagree  Accuracy: 0/106
Class: no_stance  Accuracy: 0/77
Class: not_relevant  Accuracy: 179/235
Total accuracy: 55.8333%

Temporary best.

Epoch 2:
Class: agree  Accuracy: 148/182
Class: disagree  Accuracy: 42/106
Class: no_stance  Accuracy: 4/77
Class: not_relevant  Accuracy: 183/235
Total accuracy: 62.8333%

Temporary best.

Epoch 3:
Class: agree  Accuracy: 71/182
Class: disagree  Accuracy: 68/106
Class: no_stance  Accuracy: 20/77
Class: not_relevant  Accuracy: 199/235
Total accuracy: 59.6667%

Epoch 4:
Class: agree  Accuracy: 141/182
Class: disagree  Accuracy: 60/106
Class: no_stance  Accuracy: 20/77
Class: not_relevant  Accuracy: 170/235
Total accuracy: 65.1667%

Temporary best.

Epoch 5:
Class: agree  Accuracy: 124/182
Class: disagree  Accuracy: 62/106
Class: no_stance  Accuracy: 21/77
Class: not_relevant  Accuracy: 186/235
Total accuracy: 65.5000%

Temporary best.

Epoch 6:
Class: agree  Accuracy: 101/182
Class: disagree  Accuracy

<All keys matched successfully>

In [ ]:
df_tcs = pd.read_json("final_test_candidates.json", orient='records', dtype=False)
print(df_tcs.shape)
df_tcs.head()

(1211, 2)


,tweet_id,m_id
0,1344676492102479875,10
1,1340430332231880705,3
2,1340032994254475268,3
3,1340020067476582404,10
4,1339941977174003714,3


In [ ]:
df_tts2 = pd.read_json("final_test_tweets.json", orient='records', dtype=False)
df_tts2.rename(columns={'id':'tweet_id'}, inplace = True)
print(df_tts2.shape)
df_tts2.head()

(1012, 2)


,tweet_id,text
0,1339950458480316416,COVID-19 was for introduced to gain control ov...
1,1344547117294096384,The Covid-19 vaccine will not only save lives ...
2,1340161792782188545,"@kxly4news So, a person would have equivalent ..."
3,1340803627926118400,If there is another more deadly strain of COVI...
4,1340060162967650305,"Of cooooorse they aren't URL QT: ""There are 10..."


In [ ]:
df2 = pd.merge(df_tcs, df_tts2, how='left', on=['tweet_id'])
print(df2.shape)
df2.head()

(1211, 3)


,tweet_id,m_id,text
0,1344676492102479875,10,"Just so y'all know, it's possible that the vac..."
1,1340430332231880705,3,@JCope222 Vaccine is untested for safety. Peop...
2,1340032994254475268,3,@catturd2 Why would antibodies formed from rec...
3,1340020067476582404,10,I started thinking about the long term effects...
4,1339941977174003714,3,I like vaccine herd immunity over infection he...


In [ ]:
df2['input'] = df2['m_id'].map(lambda x: discretize(m_id_dict[x], len(m_id_dict)) + " ") + df2['text']
df2['label'] = 0
df2.head()

,tweet_id,m_id,text,input,label,m_label
0,1344676492102479875,10,"Just so y'all know, it's possible that the vac...","0 1 0 0 0 0 0 Just so y'all know, it's possibl...",0,disagree
1,1340430332231880705,3,@JCope222 Vaccine is untested for safety. Peop...,0 0 0 0 0 1 0 @JCope222 Vaccine is untested fo...,0,agree
2,1340032994254475268,3,@catturd2 Why would antibodies formed from rec...,0 0 0 0 0 1 0 @catturd2 Why would antibodies f...,0,agree
3,1340020067476582404,10,I started thinking about the long term effects...,0 1 0 0 0 0 0 I started thinking about the lon...,0,no_stance
4,1339941977174003714,3,I like vaccine herd immunity over infection he...,0 0 0 0 0 1 0 I like vaccine herd immunity ove...,0,no_stance


In [ ]:
encoded_data_pred = tokenizer.batch_encode_plus(
    df2.input.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    truncation=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_pred = encoded_data_pred['input_ids']
attention_masks_pred = encoded_data_pred['attention_mask']
labels_pred = torch.tensor(df2.label.values)  # pretend to be all 0s. Proved that labels does not change results.

dataset_pred = TensorDataset(input_ids_pred, attention_masks_pred, labels_pred)

In [ ]:
dataloader_prediction = DataLoader(dataset_pred, 
                              sampler=SequentialSampler(dataset_pred),
                              batch_size=1)

In [ ]:
def predict(dataloader_val):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        logits = outputs[1]
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    predictions = np.concatenate(predictions, axis=0)
            
    return np.argmax(predictions, axis=1).flatten()

In [ ]:
test_predictions = predict(dataloader_prediction)
df2['label'] = pd.Series(test_predictions).values
print(df2.shape)
df2.head()

(1211, 6)


,tweet_id,m_id,text,input,label,m_label
0,1344676492102479875,10,"Just so y'all know, it's possible that the vac...","0 1 0 0 0 0 0 Just so y'all know, it's possibl...",1,disagree
1,1340430332231880705,3,@JCope222 Vaccine is untested for safety. Peop...,0 0 0 0 0 1 0 @JCope222 Vaccine is untested fo...,0,agree
2,1340032994254475268,3,@catturd2 Why would antibodies formed from rec...,0 0 0 0 0 1 0 @catturd2 Why would antibodies f...,3,agree
3,1340020067476582404,10,I started thinking about the long term effects...,0 1 0 0 0 0 0 I started thinking about the lon...,0,no_stance
4,1339941977174003714,3,I like vaccine herd immunity over infection he...,0 0 0 0 0 1 0 I like vaccine herd immunity ove...,3,no_stance


In [ ]:
df2['m_label'] = df2.label.replace(label_dict_inverse)
df2.head()

,tweet_id,m_id,text,input,label,m_label
0,1344676492102479875,10,"Just so y'all know, it's possible that the vac...","0 1 0 0 0 0 0 Just so y'all know, it's possibl...",1,disagree
1,1340430332231880705,3,@JCope222 Vaccine is untested for safety. Peop...,0 0 0 0 0 1 0 @JCope222 Vaccine is untested fo...,0,agree
2,1340032994254475268,3,@catturd2 Why would antibodies formed from rec...,0 0 0 0 0 1 0 @catturd2 Why would antibodies f...,3,not_relevant
3,1340020067476582404,10,I started thinking about the long term effects...,0 1 0 0 0 0 0 I started thinking about the lon...,0,agree
4,1339941977174003714,3,I like vaccine herd immunity over infection he...,0 0 0 0 0 1 0 I like vaccine herd immunity ove...,3,not_relevant


In [ ]:
import json
with open("outp/final_candidate_labels.json", mode='w', encoding='utf-8') as f:
    for elem in json.loads(df2[['tweet_id', 'm_id', 'm_label']].to_json(orient='records')):
        line = json.dumps(elem) + "\n"
        f.write(line)